In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Datasets for cost analysis.csv',index_col=1)
data = data.drop('Unnamed: 0',axis=1)

In [3]:
class chrom(object):
    def __init__(self,total_protein_conc,purity_init,target_scale):
        self.target_scale = target_scale
        # make an initial guess of the necessary total bed volume in L
        if target_scale < 500:
            self.total_BV = 7 #L
        elif target_scale < 5000:
            self.total_BV = 80 #L 
        else:
            self.total_BV = 800 #L
        
        self.resin_cost = 1840 #$/L
        self.binding_cap=0.1 #kg/L
        self.resin_lifetime=100 #cycles
        self.flowrate=150 #cm/hr
        self.equil_BVs = 6 #bed volumes
        self.wash_BVs = 5  #bed volumes
        self.elution_BVs = 4 #bed volumes
        self.regen_BVs = 4 #bed volumes
        self.buffer_base_cost = 3 #$/L
        self.buffer_base_scale =200 # kg/yr plant throughput
        self.scaling_factor = -0.56
        self.contaminant_binding = 0.5 #fraction of contaminants to account for in column sizing
        self.product_binding =1 #fraction of product to account for in column sizing
        self.hcp_lrv = 1.03 #log10 reduction value in host cell proteins
        self.slack =4 # slack time per cycle, hrs
        self.frac_year = 0.85 # fraction of year worked
        self.height = 0.3 #column height in m
        self.bed_frac = 0.85 # fraction of column height packed with resin
        self.max_diam = 2 # max column diameter in m
        self.install_frac = 0.05 # fraction of column purchase cost that installation costs
        self.maint_frac = 0.1 #fraction of column purchase cost that annual maintenance costs
        self.ito_frac = 0.08 #fraction of column purchase cost that insurance, taxes and overhead costs
        self.qc_frac = 0.25 # fraction of labor hours added on for QC
        self.labor_rate = 80.5 #$/hr of labor
        self.validation_time = 2 #hrs per cycle for column validation
        self.sanitization_time = 4 # hrs per cycle for cleaning with 0.5 M NaOH
        
        self.total_protein_conc = total_protein_conc
        self.purity_init = purity_init
        self.yield_frac = 0.95
    
    def solve(self):
        self.kg_bound_per_kg_yielded = ((self.contaminant_binding*(1-self.purity_init))+(self.purity_init*self.product_binding))/(self.product_binding*self.purity_init*self.yield_frac)
        
        self.replacements_per_cycle = (1/self.resin_lifetime)
        
        self.liter_cycles_per_kg_total_bound = (1/self.binding_cap)

        self.bed_height_cm = self.height*self.bed_frac*100

        self.max_BV_per_col=1000*((self.max_diam/2)**2)*np.pi*self.height*self.bed_frac

        self.num_cols = np.ceil(self.total_BV/self.max_BV_per_col)

        self.BV_per_col = self.total_BV/self.num_cols

        self.kg_per_col_per_cycle = self.BV_per_col/(self.kg_bound_per_kg_yielded*self.liter_cycles_per_kg_total_bound)

        self.loading_BVs = 1000*self.binding_cap/self.total_protein_conc

        self.sanitization_BVs = self.sanitization_time*self.flowrate/self.bed_height_cm
        self.flowrate_in_BVs= self.height*self.bed_frac*100/self.flowrate
        self.hours_per_cycle = (self.sanitization_time+self.validation_time \
                                            + \
                                                (self.flowrate_in_BVs \
                                                 *\
                                                    (\
                                                    self.equil_BVs+self.wash_BVs+\
                                                     self.elution_BVs+self.regen_BVs+\
                                                     self.loading_BVs\
                                                    )\
                                                )\
                               )

        self.operator_hrs_per_cycle= self.hours_per_cycle*self.num_cols

        self.col_diam = 2*np.sqrt(
            (self.BV_per_col/1000)
            /
            (self.bed_height_cm*0.01*np.pi)
                                                )
        
        self.total_L_per_col = ((0.5*self.col_diam)**2)*self.height*np.pi*1000

        self.cycles_per_day =  1/(
            (self.hours_per_cycle+self.slack)
            /24
        )

        self.kg_per_yr = self.num_cols*self.kg_per_col_per_cycle*self.cycles_per_day*self.frac_year*365

        self.buffer_price =(
            self.buffer_base_cost
            *((self.kg_per_yr/self.buffer_base_scale)**self.scaling_factor)
        )
        
        if self.buffer_price > 10:
            self.buffer_price = 10

        self.NaOH_price = self.buffer_price

        self.col_PC = (self.num_cols*
                              ((self.total_L_per_col*941)+147854)
        )

        self.unlisted_PC = self.col_PC/4

        self.total_PC = self.unlisted_PC+self.col_PC

        self.installation = (
            (self.install_frac*self.col_PC)
            +
            (0.5*self.unlisted_PC)
        )

        self.DC = (self.installation+
                                (self.total_PC*1.88)
                                )
        self.IC = self.DC*0.6

        self.DFC = (self.DC+self.IC+\
                        ((self.IC+self.DC)*0.15)\
                       )


        self.maint_per_yr = self.total_PC*self.maint_frac
        self.ito_per_yr = self.ito_frac*self.DFC


        self.maint_per_kg = self.maint_per_yr/self.kg_per_yr
        self.ito_per_kg = self.ito_per_yr/self.kg_per_yr
        self.labor_per_kg = self.operator_hrs_per_cycle*self.labor_rate/(self.kg_per_col_per_cycle*self.num_cols)
        self.QC_per_kg=self.qc_frac*self.labor_per_kg
        self.buffer_per_kg = (self.buffer_price*\
                              ((self.equil_BVs+self.wash_BVs+self.elution_BVs+self.regen_BVs)\
                                   *self.BV_per_col))/self.kg_per_col_per_cycle\

        self.resin_per_kg= self.liter_cycles_per_kg_total_bound*self.replacements_per_cycle*self.kg_bound_per_kg_yielded*self.resin_cost
        self.NaOH_per_kg = self.NaOH_price*self.sanitization_BVs*self.BV_per_col/self.kg_per_col_per_cycle

        self.total_cost_per_kg = sum([self.maint_per_kg,self.ito_per_kg,
        self.labor_per_kg,self.QC_per_kg,self.buffer_per_kg,self.resin_per_kg,
        self.NaOH_per_kg])

        self.total_cost_per_kg_per_HCP_lrv = self.total_cost_per_kg/self.hcp_lrv
        
    def adjust(self):
        if self.kg_per_yr != self.target_scale:
            self.total_BV = self.total_BV*(self.target_scale/self.kg_per_yr)
            

In [4]:
class phase(object):
    def __init__(self,df,prod_conc,hcp_lrv,yield_frac,react_temp,react_time,target_scale):
        self.total_tank_cap = 100 # L
        self.df = df
        self.target_scale = target_scale
        
        self.WFI_base_price=3 #$/L
        self.WFI_base_scale=200 #kg total plant capacity per yr
        self.frac_year=0.85 # fraction of year worked
        self.slack=4 # slack hours per cycle
        self.scaling_factor = -0.56
        self.install_frac =0.3 #fraction of tank purchase cost that installation costs
        self.maint_frac =0.1 # fraction of tank purchase cost that annual maintenance costs
        self.ito_frac=0.08 # fraction of tank purchase cost that insurance, taxes and overhead cost
        self.qc_frac=0.25 #fraction of labor hours added on for QC
        self.labor_rate=80.50 #$/hr of labor
        self.tank_usage_frac= 0.9 #tank volumetric usage fraction, to account for headspace
        self.validation_time=0.5 #hrs per cycle for tank validation
        self.caustic_time=0.5 #caustic agent CIP time per cycle, hrs
        self.acid_time=0.5 #acid agent CIP time per cycle, hrs
        self.WFI_time=10.0/60 #WFI CIP time per cycle, hrs
        self.SIP_time=1 # steam in place time per cycle, hrs
        self.CIP_fluid_flowrate=420 #L/hr per m of tank circumference
        self.WFI_flowrate=840 #L/hr per m of tank circumference
        self.height_to_diam=3 # tank height to diameter ratio
        self.heating_cooling_rate=1 #Deg C per min
        self.heat_cap=4182 # J/kg per deg C, specific heat capacity of water
        self.electrical_price=0.1 #$/kW-hr
        self.initial_temp=25 #Deg C
        
        self.prod_conc = prod_conc
        self.hcp_lrv = hcp_lrv
        self.yield_frac = yield_frac
        self.react_temp = react_temp
        self.react_time = react_time

    def solve(self):
    
        self.num_tanks = np.ceil((self.total_tank_cap/self.tank_usage_frac)/80000)

        self.total_vol_per_tank = self.total_tank_cap/(self.num_tanks*self.tank_usage_frac)

        self.hrs_per_cycle = sum([self.validation_time,self.caustic_time,self.acid_time,
                                  self.WFI_time,self.react_time,self.SIP_time])
        self.operator_hrs_per_cycle = self.hrs_per_cycle*self.num_tanks

        self.kg_per_L_per_cycle = self.prod_conc*self.yield_frac/1000
        self.kg_per_tank_per_cycle = self.kg_per_L_per_cycle*self.total_tank_cap/self.num_tanks
        self.cycles_per_day = 1/((self.hrs_per_cycle+self.slack)/24)

        self.kg_per_yr = self.kg_per_L_per_cycle*self.total_tank_cap*self.cycles_per_day*self.frac_year*365

        self.WFI_price = (self.WFI_base_price\
                                  *((self.kg_per_yr/self.WFI_base_scale)\
                                   **self.scaling_factor)\
        )
        
        if self.WFI_price > 10:
            self.WFI_price = 10

        self.NaOH_price = self.WFI_price
        self.H3PO4_price = self.WFI_price


        self.tank_diam = 2*(np.cbrt((\
            (self.total_vol_per_tank/1000.0)\
            /(self.height_to_diam*2*np.pi)\
                                        )))

        self.tank_circ = self.tank_diam*np.pi

        self.heating_cooling_time = abs(self.initial_temp-self.react_temp)/60
        self.kW_hrs_per_L_per_cylce = (\
            (self.heat_cap*self.heating_cooling_rate/60)\
            *self.heating_cooling_time/1000\
        )

        self.tank_PC = (self.num_tanks\
                            *(\
                                ((self.total_vol_per_tank**2)*(-3.86*(10**-5)))\
                                +(self.total_vol_per_tank*6.32)\
                                +253144\
                           ))


        self.unlisted_PC = self.tank_PC/4

        self.total_PC = self.unlisted_PC + self.tank_PC

        self.installation_cost = (
            (self.install_frac*self.tank_PC)\
            +\
            (0.5*self.unlisted_PC)\
        )

        self.DC = (self.installation_cost+\
                                (self.total_PC*1.88)\
                                )
        self.IC = self.DC*0.6

        self.DFC = (self.DC+self.IC+\
                        ((self.IC+self.DC)*0.15)\
                       )


        self.maint_per_yr = self.total_PC*self.maint_frac
        self.ito_per_yr = self.ito_frac*self.DFC


        self.maint_cost_per_kg = self.maint_per_yr/self.kg_per_yr
        self.ito_cost_per_kg = self.ito_per_yr/self.kg_per_yr
        self.labor_cost_per_kg = self.operator_hrs_per_cycle*self.labor_rate/(self.kg_per_tank_per_cycle*self.num_tanks)
        self.QC_cost_per_kg = self.qc_frac*self.labor_cost_per_kg

        self.NaOH_cost_per_kg = self.NaOH_price*self.tank_circ*self.CIP_fluid_flowrate*self.caustic_time/self.kg_per_tank_per_cycle
        self.H3PO4_cost_per_kg = self.H3PO4_price*self.tank_circ*self.CIP_fluid_flowrate*self.acid_time/self.kg_per_tank_per_cycle
        self.WFI_cost_per_kg = self.WFI_price*self.tank_circ*self.WFI_flowrate*self.WFI_time/self.kg_per_tank_per_cycle

        self.heating_cooling_cost_per_kg = self.electrical_price*self.kW_hrs_per_L_per_cylce/self.kg_per_L_per_cycle

        materials_kg_per_yr = []
        materials_price_per_kg = []
        materials_costs_per_kg_product = []
        materials_kg_per_kg_product = []
        for j in [1,2,3,4]:
            if not pd.isnull(self.df['Material {} g/L'.format(str(j))]):
                materials_kg_per_yr.append(self.df['Material {} g/L'.format(str(j))]*self.total_tank_cap*self.cycles_per_day*self.frac_year*365/1000)
            if pd.isnull(self.df['Material {} g/L'.format(str(j))]) and not pd.isnull(self.df['Material {} kg/kg product yielded'.format(str(j))]):
                materials_kg_per_yr.append(self.df['Material {} kg/kg product yielded'.format(str(j))]*self.kg_per_yr)
            if pd.isnull(self.df['Material {} g/L'.format(str(j))]) and pd.isnull(self.df['Material {} kg/kg product yielded'.format(str(j))]):
                materials_kg_per_yr.append(np.nan)

            materials_price_per_kg.append((\
            self.df['Material {} base price'.format(str(j))]\
            *((materials_kg_per_yr[j-1]/self.df['Material {} base kg'.format(str(j))])**self.scaling_factor)\
        ))

            materials_kg_per_kg_product.append(materials_kg_per_yr[j-1]/self.kg_per_yr)
            materials_costs_per_kg_product.append(materials_kg_per_kg_product[j-1]*materials_price_per_kg[j-1])
    
            self.materials_cost_per_kg = np.nansum(materials_costs_per_kg_product)
            
            self.total_cost_per_kg = sum([self.maint_cost_per_kg,self.ito_cost_per_kg,\
                                         self.labor_cost_per_kg,self.QC_cost_per_kg,\
                                         self.NaOH_cost_per_kg,self.H3PO4_cost_per_kg,\
                                         self.WFI_cost_per_kg,self.heating_cooling_cost_per_kg,\
                                         self.materials_cost_per_kg])
            self.total_cost_per_kg_per_HCP_lrv = self.total_cost_per_kg/self.hcp_lrv
                                          
    def adjust(self):
        if self.kg_per_yr != self.target_scale:
            self.total_tank_cap = self.total_tank_cap*(self.target_scale/self.kg_per_yr)                  

In [7]:
for i in data.index:
    if pd.isnull(data.at[i,'Initial product concentration (g/L)']):
        data.at[i,'Chrom cost'] = np.nan
        continue
        
    total_protein_conc = data.at[i,'Initial product concentration (g/L)']/(data.at[i,'Purity initial (%)']/100)
    purity_init = data.at[i,'Purity initial (%)']/100
    
    prod_conc = data.at[i,'Initial product concentration (g/L)']
    hcp_lrv = data.at[i,'HCP lrv']
    yield_frac = data.at[i,'Yield (%)']/100
    react_temp = data.at[i,'Temp']
    react_time = data.at[i,'Time (hr)']
    
    
    for target_scale in [10,100,1000,10000]:
        x = chrom(total_protein_conc,purity_init,target_scale)
        x.solve()
        x.adjust()
        x.solve()
        
        data.at[i,'Chrom cost per kg {} kg/yr scale'.format(target_scale)] = x.total_cost_per_kg
        data.at[i,'Chrom cost per kg per HCP lrv {} kg/yr scale'.format(target_scale)] =x.total_cost_per_kg_per_HCP_lrv
        data.at[i,'Chrom facility costs per kg at {} kg/yr'.format(target_scale)] = x.maint_per_kg+x.ito_per_kg
        data.at[i,'Chrom labor costs per kg at {} kg/yr'.format(target_scale)] = x.labor_per_kg+x.QC_per_kg
        data.at[i,'Chrom direct materials costs per kg at {} kg/yr'.format(target_scale)] = x.resin_per_kg+x.buffer_per_kg
        data.at[i,'Chrom indirect materials costs per kg at {} kg/yr'.format(target_scale)] = x.NaOH_per_kg
        
        
        y = phase(data.loc[i,:],prod_conc,hcp_lrv,yield_frac,react_temp,react_time,target_scale)
        y.solve()
        y.adjust()
        y.solve()
        
        data.at[i,'Phase cost per kg {} kg/yr scale'.format(target_scale)] = y.total_cost_per_kg
        data.at[i,'Phase cost per kg per HCP lrv {} kg/yr scale'.format(target_scale)] =y.total_cost_per_kg_per_HCP_lrv
        data.at[i,'Phase facility costs per kg at {} kg/yr'.format(target_scale)] = y.maint_cost_per_kg+y.ito_cost_per_kg
        data.at[i,'Phase labor costs per kg at {} kg/yr'.format(target_scale)] = y.labor_cost_per_kg+y.QC_cost_per_kg
        data.at[i,'Phase indirect materials costs per kg at {} kg/yr'.format(target_scale)] = (y.NaOH_cost_per_kg + 
                                                                                      y.WFI_cost_per_kg+
                                                                                      y.H3PO4_cost_per_kg)
        
        data.at[i,'Phase direct materials costs per kg at {} kg/yr'.format(target_scale)] = y.materials_cost_per_kg                                                                             
        data.at[i,'Phase utilities costs per kg at {} kg/yr'.format(target_scale)] = y.heating_cooling_cost_per_kg
    


In [6]:
data.to_csv('Economic analysis.csv')

In [8]:
data

,Recipe,Initial product concentration (g/L),Purity initial (%),HCP lrv,Yield (%),Material 1 g/L,Material 1 kg/kg product yielded,Material 2 g/L,Material 2 kg/kg product yielded,Material 3 g/L,...,Phase direct materials costs per kg at 10000 kg/yr,Phase utilities costs per kg at 10000 kg/yr,Chrom direct materials costs per kg at 10 kg/yr,Chrom indirect materials costs per kg at 10 kg/yr,Chrom direct materials costs per kg at 100 kg/yr,Chrom indirect materials costs per kg at 100 kg/yr,Chrom direct materials costs per kg at 1000 kg/yr,Chrom indirect materials costs per kg at 1000 kg/yr,Chrom direct materials costs per kg at 10000 kg/yr,Chrom indirect materials costs per kg at 10000 kg/yr
Index,,,,,,,,,,,,,,,,,,,,,
32,"Intact leaves, 70 C, 15 min",NaN,0.10,1.40,51.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,"Intact leaves, 70 C, 15 min",NaN,3.30,1.88,83.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,"Hemolysed blood, 25% v/v ethanol, 0.6% v/v chl...",11.97,17.10,1.05,42.0,197.2500,NaN,8.94000,NaN,NaN,...,4759.750544,0.346601,7511.123423,8480.437420,3691.888195,3750.734351,1497.348408,1033.038021,892.921961,284.522297
35,"2.3 g sodium octanoate per 100 g protein, 2 mM...",2.43,48.60,0.87,88.0,NaN,0.053779,0.58448,NaN,NaN,...,101.184448,2.444585,3353.718865,3786.517856,1648.429188,1674.704015,668.566514,461.251787,398.689924,127.039291
42,"130 mM NaCl, 25 mM Na acetate\nbuffer pH 5.0 a...",0.40,50.00,1.74,96.0,7.5972,NaN,2.05075,NaN,0.16,...,393.585177,0.000000,3290.526316,3715.170279,1617.368611,1643.148354,655.969029,452.560636,391.177597,124.645549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,"Precipitation: 1 mg/mL mAb, 0.41 mg/mL HCPs, 5...",1.00,70.92,1.44,100.0,28.8186,NaN,150.00000,NaN,NaN,...,362.818109,0.000000,2643.432778,2984.569017,1299.307403,1320.017469,526.970420,363.563000,314.251151,100.133565
282,"Precipitation: 1 mg/mL mAb, 0.41 mg/mL HCPs, 5...",1.00,70.92,1.44,98.4,28.8186,NaN,150.00000,NaN,NaN,...,365.402162,0.000000,2643.432778,2984.569017,1299.307403,1320.017469,526.970420,363.563000,314.251151,100.133565
283,"Precipitation: 1 mg/mL mAb, 0.41 mg/mL HCPs, 5...",1.00,70.92,1.62,99.3,28.8186,NaN,150.00000,NaN,NaN,...,363.941254,0.000000,2643.432778,2984.569017,1299.307403,1320.017469,526.970420,363.563000,314.251151,100.133565
